# M1 Regular Expression 

* **Objective** : looking at two distinct point of NFT price behavior within the same collection(BAYC)

* **PARTA**: Price First Time Sold Against Characteristics of NFT
  * Consider the traits and the orginal designed purpose(ie. planning on selling it to whom) is the NFT's initial characteristics that affect the initial price at day 1 when it is published. 
  * Time fixed effect can be applied here because the NFT market is highly volatile and often influenced by market sentiment. Meanwhile, our data spans from a wide range of time so it should make sense in using this. 
  * $$\text{Price}_i = \beta_i \times \text{NFTCharacteristics}_i + \text{Time\_1\_Sale} + \epsilon_i$$

  * Dependent Variable: Price at first sale.
  * Key Regressor: NFT characteristics: `Background` `Fur` etc. 
  * The time when the first sale happeend could be a useful indicator of the first_sale price


Rationale:
Since we lack minting data, using the sale date is a valid alternative. The fixed effects for sale month will help absorb time-specific factors (e.g., market sentiment, liquidity, crypto market trends) that may otherwise bias the effect of rarity on the price. This approach isolates the within–month variation in price that is explained by differences in NFT characteristics.
  
* **PARTB**: Percentage Change in Price as Function of Seller Profile
  * Dependent Variable: Percentage change in price (e.g., from first sale to a subsequent sale).
  * Key Regressors: Seller profile indicators, such as the number of NFTs they own and the number of NFTs they sell, which may serve as proxies for market power or reputation.

Rationale:
This regression is intended to understand how variations in seller behavior or profile attributes are associated with the evolution (or reappraisal) of NFT prices. By focusing on the percentage change,   capture the dynamic adjustment—potentially related to liquidity, strategic behavior, or reputation effects—that occurs after the NFT’s first sale.

$$
\log P_i^{(1)}
   \;=\;
   \alpha
   \;+\;
   \mathbf X_i^{\top}\boldsymbol\beta
   \;+\;
   \delta_{m(i)}
source
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold
import statsmodels.formula.api as smf

# --- Clean the data ONCE, before splitting ---
trait_cols = ['Background','Clothes','Earring','Eyes','Fur','Hat','Mouth']
df_clean = df.copy()
df_clean[trait_cols] = df_clean[trait_cols].fillna('Unknown')
df_clean = df_clean[df_clean['price_1_sale'].notna() & (df_clean['price_1_sale'] > 0)].copy()
df_clean['price_1_sale'] = df_clean['price_1_sale'].clip(",
,
,
,
,
,
,
1
,
,
,
,
,
,
,
,
,
,
,

## PARTA: OLS on first price sale with traits only

In [1]:
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS
# poolols
from linearmodels.panel import PooledOLS
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import statsmodels.api as sm

# Load and prepare data

df1 = pd.read_csv('df_table1.csv')
df3 = pd.read_csv('df_table3.csv')

# merge df1 and df3 on token_id
df = pd.merge(df1, df3, on='token_id', how='left')


cols_to_keep = ['token_id','price_1_sale','Background','Clothes','Earring','Eyes','Fur','Hat','Mouth']

df = df[cols_to_keep]

# winsorize price_1_sale at 1% and 99% and log transform 
df['price_1_sale'] = df['price_1_sale'].clip(lower=df['price_1_sale'].quantile(0.01), upper=df['price_1_sale'].quantile(0.99))
df['log_price_1_sale'] = np.log(df['price_1_sale'])
df = df.drop(columns=['price_1_sale']) # drop original price_1_sale column

import statsmodels.formula.api as smf

# this will create a dummy for each level of `Mouth` (and drop one as the reference)
formula = (
    'log_price_1_sale ~ '
    'C(Background) + '
    'C(Clothes) + '
    'C(Earring) + '
    'C(Eyes) + '
    'C(Fur) + '
    'C(Hat) + '
    'C(Mouth)'
)
mdl = smf.ols(formula, data=df).fit()
print(mdl.summary())


                            OLS Regression Results                            
Dep. Variable:       log_price_1_sale   R-squared:                       0.153
Model:                            OLS   Adj. R-squared:                  0.066
Method:                 Least Squares   F-statistic:                     1.760
Date:                Mon, 15 Dec 2025   Prob (F-statistic):           9.37e-08
Time:                        19:35:55   Log-Likelihood:                -3274.6
No. Observations:                1710   AIC:                             6869.
Df Residuals:                    1550   BIC:                             7740.
Df Model:                         159                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

### RMSE 12/15/25

In [3]:
# RMSE 12/15/25 — clean once, then compute in-sample, test, and CV RMSE
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold
import statsmodels.formula.api as smf

# Start from the already prepared df (log_price_1_sale exists, price_1_sale dropped)
df_model = df.copy()
trait_cols = ['Background','Clothes','Earring','Eyes','Fur','Hat','Mouth']

# Fill missing categorical levels to avoid row drops in OLS/patsy
df_model[trait_cols] = df_model[trait_cols].fillna('Unknown')

# Drop any remaining missing/inf in target
mask_finite = np.isfinite(df_model['log_price_1_sale'])
df_model = df_model[mask_finite].copy()

# 1) In-sample RMSE on estimation sample
mdl_full = smf.ols(formula, data=df_model).fit()
print('df_model rows:', len(df_model), '| mdl_full.nobs:', int(mdl_full.nobs))
rmse_in = np.sqrt(mean_squared_error(mdl_full.model.endog, mdl_full.fittedvalues))
print(f'In-sample RMSE (log): {rmse_in:.4f}')

# 2) Test RMSE (hold-out)
train_df, test_df = train_test_split(df_model, test_size=0.2, random_state=87)
mdl_tr = smf.ols(formula, data=train_df).fit()
y_te = test_df['log_price_1_sale'].to_numpy()
y_te_hat = mdl_tr.predict(test_df)
rmse_test = np.sqrt(mean_squared_error(y_te, y_te_hat))
print(f'Test RMSE (log): {rmse_test:.4f}')

# 3) K-fold CV RMSE
kf = KFold(n_splits=10, shuffle=True, random_state=87)
rmses = []
for tr_idx, te_idx in kf.split(df_model):
    tr = df_model.iloc[tr_idx]
    te = df_model.iloc[te_idx]
    m = smf.ols(formula, data=tr).fit()
    pred = m.predict(te)
    rmses.append(np.sqrt(mean_squared_error(te['log_price_1_sale'].to_numpy(), pred)))

rmses = np.array(rmses)
print('CV RMSE per fold (log):', rmses)
print(f'Mean CV RMSE (log): {rmses.mean():.4f}')
print(f'Std  CV RMSE (log): {rmses.std():.4f}')


print(f"RMSE (log), mean across folds: {rmses.mean():.2f}")


df_model rows: 9254 | mdl_full.nobs: 9254
In-sample RMSE (log): 1.7037
Test RMSE (log): 1.6795
CV RMSE per fold (log): [1.66742902 1.68762944 1.76195204 1.74312454 1.73242147 1.73653512
 1.77630279 1.7524836  1.74017933 1.74597352]
Mean CV RMSE (log): 1.7344
Std  CV RMSE (log): 0.0313
RMSE (log), mean across folds: 1.73


### Largest Coeficients

In [17]:
import pandas as pd

# 1) Build a DataFrame of all coefficients and p-values
results = pd.DataFrame({
    'coef':   mdl.params,
    'pvalue': mdl.pvalues
})

# 2) (Optional) drop the intercept row
if 'Intercept' in results.index:
    results = results.drop('Intercept')

# 3) Keep only those with p < 0.1
sig = results[results['pvalue'] < 0.1]

# 4a) Get the largest positive coefficients
largest = sig.sort_values('coef', ascending=False)
print("Largest coefficients (p<0.1):")
print(largest.head(5))

# 4b) Get the most negative coefficients
smallest = sig.sort_values('coef', ascending=True)
print("\nSmallest coefficients (p<0.1):")
print(smallest.head(5))


Largest coefficients (p<0.1):
                             coef    pvalue
C(Eyes)[T.Blue Beams]    3.706082  0.041351
C(Mouth)[T.Bored Kazoo]  1.903832  0.000478
C(Fur)[T.Solid Gold]     1.757636  0.011752
C(Eyes)[T.Laser Eyes]    1.675571  0.017245
C(Mouth)[T.Bored Pizza]  1.667729  0.014401

Smallest coefficients (p<0.1):
                                     coef    pvalue
C(Clothes)[T.Sleeveless Logo T] -1.416980  0.037227
C(Clothes)[T.Lumberjack Shirt]  -1.239451  0.053279
C(Clothes)[T.Navy Striped Tee]  -1.233548  0.051422
C(Clothes)[T.Biker Vest]        -1.153996  0.070901
C(Clothes)[T.Lab Coat]          -1.125785  0.083294


##### Reference Level

In [18]:
import numpy as np
pct = (np.exp(3.71) - 1) * 100
print(f"{pct:.1f}%")    # → about 3970.7%


3985.4%


In [5]:
import re

def find_reference(var):
    # all observed levels, sorted
    levels = sorted(df[var].dropna().unique())
    # the levels that appear in your params as T.<level>
    treated = [
        re.search(rf'C\({var}\)\[T\.(.*?)\]', name).group(1)
        for name in mdl.params.index
        if name.startswith(f'C({var})')
    ]
    # the one that’s in levels but not in treated is the reference
    ref = [lvl for lvl in levels if lvl not in treated]
    return ref[0] if ref else None

for cat in ['Background','Clothes','Earring','Eyes','Fur','Hat','Mouth']:
    print(f"{cat!r} reference level:", find_reference(cat))


'Background' reference level: Aquamarine
'Clothes' reference level: Admirals Coat
'Earring' reference level: Cross
'Eyes' reference level: 3d
'Fur' reference level: Black
'Hat' reference level: Army Hat
'Mouth' reference level: Bored


# Fixed Effect for PartA on Price_1_Sale

In [3]:
import os, glob, json, random
import numpy as np
import pandas as pd
import polars as pl
from sklearn.model_selection   import train_test_split
from sklearn.preprocessing     import StandardScaler
from sklearn.linear_model      import ElasticNetCV
from sklearn.metrics           import mean_squared_error, mean_absolute_error
import statsmodels.api as sm 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")
import warnings
from sklearn.exceptions import ConvergenceWarning
# ignore runtime overflows from coordinate descent
warnings.filterwarnings("ignore", category=RuntimeWarning)
# ignore convergence warnings 
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# read in df_table1 
df_table1 = pl.read_csv(
    "df_table1.csv",
    columns=["token_id", "time_1_sale", "price_1_sale"]
).with_columns([
    # Convert UNIX seconds → datetime → format "YYYY-MM"
    pl.col("time_1_sale")
      .cast(pl.Int64)
      .mul(1000)  # Convert seconds to milliseconds
      .cast(pl.Datetime(time_unit="ms"))
      .dt.strftime("%Y-%m")
      .alias("sale_1_month")
])

# read in df_table3 
df_table3 = pl.read_csv("df_table3.csv")


# merge df_table1 and df_table3 on token_id

df_reg1 = df_table1.join(
    df_table3,
    left_on="token_id",
    right_on="token_id",
    how="left",
).with_columns([
    # Convert UNIX seconds → datetime → format "YYYY-MM"
    pl.col("time_1_sale")
      .cast(pl.Int64)
      .mul(1000)  # Convert seconds to milliseconds
      .cast(pl.Datetime(time_unit="ms"))
      .dt.strftime("%Y-%m")
      .alias("sale_1_month")
])

# df_reg1 drop time_1_sale 

df_partA = df_reg1.drop("time_1_sale")

# categorical columns
cat_cols = ["sale_1_month","Background", "Clothes","Earring", "Eyes","Fur", "Hat","Mouth"]

# One-hot encode categorical features and drop one level per feature (base level)
df_partA = df_partA.to_pandas()
df_partA = pd.get_dummies(df_partA, columns=cat_cols, drop_first=True).copy() # drop_first=True drops the first level of each categorical feature

df_partA.drop(columns=['rarity.rank'], inplace=True) # any analysis on rarity.rank is leakage because it is less likely to stay the same across times

for col in df_partA.columns:
    if df_partA[col].dtype == bool:
        df_partA[col] = df_partA[col].astype("int8")
        
# log+1 on price_1_sale
df_partA['log_price_1'] = np.log1p(df_partA['price_1_sale'])
df_partA.drop(columns=['price_1_sale'], inplace=True) # drop price_1_sale

# X and Y 
drop_cols = ['token_id', 'log_price_1']
X = df_partA.drop(columns=drop_cols)
y = df_partA['log_price_1'].values

# reference levels 
reference_levels = {}
for cat in cat_cols:
    # Use df_reg1 (polars DataFrame) and convert to pandas Series for unique values
    all_levels = sorted(df_reg1[cat].to_pandas().dropna().unique())
    dummy_cols = [c for c in X.columns if c.startswith(cat + "_")]
    kept_levels = [c.split(cat + "_", 1)[1] for c in dummy_cols]
    # the reference is the one present in all_levels but not in kept_levels
    ref = [lvl for lvl in all_levels if lvl not in kept_levels]
    reference_levels[cat] = ref[0] if ref else None

print("Reference (dropped) levels:")
for cat, ref in reference_levels.items():
    print(f"  {cat}: {ref}")


# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=87)
scalar = StandardScaler()
scalar.fit(X_train)
X_train = scalar.transform(X_train)
X_test = scalar.transform(X_test)

# ElasticNetCV on ten fold cross validation 

model = ElasticNetCV(
                    l1_ratio=np.linspace(0.01, 20, 100),
                    # Reduce the upper limit of logspace, e.g., from 4 to 2 or 1
                    alphas =np.logspace(-4, 2, 50), # Changed upper limit from 4 to 2
                    cv=10, 
                    random_state=87,
                    n_jobs=4)

model.fit(X_train, y_train)

print("Best alpha: ", model.alpha_) 
print("Best l1_ratio: ", model.l1_ratio_)
print("Best score: ", model.score(X_test, y_test))


# evaluate model performance
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"R^2: {model.score(X_test, y_test):.2f}")

# non-zero coefficients
coef = pd.Series(model.coef_, index=X.columns)
coef = coef[coef != 0].sort_values(ascending=False)
coef



/var/folders/zd/cp61vmzd4179rc6wz_57mvv00000gn/T/ipykernel_45488/3573893098.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_partA['log_price_1'] = np.log1p(df_partA['price_1_sale'])


Reference (dropped) levels:
  sale_1_month: 2021-04
  Background: Aquamarine
  Clothes: Admirals Coat
  Earring: Cross
  Eyes: 3d
  Fur: Black
  Hat: Army Hat
  Mouth: Bored
Best alpha:  0.0016768329368110084
Best l1_ratio:  0.41383838383838384
Best score:  0.9144821580010601
RMSE: 0.36
MAE: 0.22
MSE: 0.13
R^2: 0.91


sale_1_month_2022-01    0.203713
sale_1_month_2022-03    0.167526
sale_1_month_2022-05    0.162278
sale_1_month_2022-04    0.159343
Fur_Solid Gold          0.119701
                          ...   
Clothes_Biker Vest     -0.015221
Eyes_Closed            -0.017993
sale_1_month_2021-07   -0.120207
sale_1_month_2021-06   -0.549966
sale_1_month_2021-05   -1.061624
Length: 201, dtype: float64

### Getting the RMSE_CV 

In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import ElasticNetCV
import numpy as np

RNG = 87

pipe = make_pipeline(
    StandardScaler(),
    ElasticNetCV(
        alphas=np.logspace(-4, 2, 50),
        l1_ratio=np.linspace(0.01, 1.0, 20),
        cv=10,
        random_state=RNG,
        n_jobs=-1
    )
)

cv = KFold(n_splits=10, shuffle=True, random_state=RNG)

# CV RMSE (one value per fold)
rmse_scores = -cross_val_score(
    pipe,
    X, y,
    cv=cv,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

print("CV RMSE per fold:", rmse_scores)
print(f"Mean CV RMSE: {rmse_scores.mean():.4f}")
print(f"Std  CV RMSE: {rmse_scores.std():.4f}")

/Users/miultra/Documents/Tech/Miniconda/miniconda3/envs/NFT_25/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.8257242955734228, tolerance: 1.184992903362089
  model = cd_fast.enet_coordinate_descent_gram(
/Users/miultra/Documents/Tech/Miniconda/miniconda3/envs/NFT_25/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25.551361718040198, tolerance: 1.184992903362089
  model = cd_fast.enet_coordinate_descent_gram(
/Users/miultra/Documents/Tech/Miniconda/miniconda3/envs/NFT_25/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.358254296349969, tolerance: 1.174709227

CV RMSE per fold: [0.39664076 0.31856206 0.291898   0.32321187 0.36578587 0.33718591
 0.30889633 0.35034178 0.3343785  0.30748436]
Mean CV RMSE: 0.3334
Std  CV RMSE: 0.0294


In [2]:
import statsmodels.api as sm

# 1) Identify which features survived the ElasticNet
selected = X.columns[model.coef_ != 0].tolist()
print(f"Features selected by ElasticNet ({len(selected)}):\n{selected}\n")

# 2) Reconstruct pandas DataFrames for train & test 
#    (we need columns for statsmodels)
X_train_df = pd.DataFrame(X_train, columns=X.columns)
X_test_df  = pd.DataFrame(X_test,  columns=X.columns)

# 3) Subset to only the selected features
X_sel_train = X_train_df[selected]
X_sel_test  = X_test_df[selected]

# 4) Add a constant term
X_sel_train_const = sm.add_constant(X_sel_train)

# 5) Fit OLS on the *training* data
ols = sm.OLS(y_train, X_sel_train_const).fit()

# 6) Print the full regression table
print(ols.summary())

# Optionally, evaluate out‐of‐sample performance on the test set
X_sel_test_const = sm.add_constant(X_sel_test, has_constant='add')
y_pred_ols = ols.predict(X_sel_test_const)

from sklearn.metrics import mean_squared_error
print("Test RMSE (OLS):", np.sqrt(mean_squared_error(y_test, y_pred_ols)))


Features selected by ElasticNet (201):
['sale_1_month_2021-05', 'sale_1_month_2021-06', 'sale_1_month_2021-07', 'sale_1_month_2021-08', 'sale_1_month_2021-09', 'sale_1_month_2021-10', 'sale_1_month_2021-11', 'sale_1_month_2021-12', 'sale_1_month_2022-01', 'sale_1_month_2022-02', 'sale_1_month_2022-03', 'sale_1_month_2022-04', 'sale_1_month_2022-05', 'sale_1_month_2022-06', 'sale_1_month_2022-07', 'sale_1_month_2022-08', 'sale_1_month_2022-09', 'sale_1_month_2022-10', 'sale_1_month_2022-11', 'sale_1_month_2022-12', 'sale_1_month_2023-01', 'sale_1_month_2023-02', 'sale_1_month_2023-03', 'sale_1_month_2023-04', 'sale_1_month_2023-05', 'sale_1_month_2023-06', 'sale_1_month_2023-07', 'sale_1_month_2023-08', 'sale_1_month_2023-09', 'sale_1_month_2023-10', 'sale_1_month_2023-11', 'sale_1_month_2023-12', 'sale_1_month_2024-01', 'sale_1_month_2024-02', 'sale_1_month_2024-03', 'sale_1_month_2024-04', 'sale_1_month_2024-05', 'sale_1_month_2024-06', 'sale_1_month_2024-07', 'sale_1_month_2024-08', 

In [3]:
import pandas as pd

# 1) Build DataFrame of coef & p-values (drop the intercept if you like)
results_df = pd.DataFrame({
    'coef':   ols.params,
    'pvalue': ols.pvalues
})
# optionally drop the constant
if 'const' in results_df.index:
    results_df = results_df.drop('const')

# 2) Filter out any with p-value > 0.1
results_df = results_df[results_df['pvalue'] <= 0.1]

# 3) Sort by magnitude of coef (largest first)
results_df = results_df.reindex(
    results_df['coef']
              .sort_values(ascending=False)
              .index
)

# 4) Reset index if you want a clean “Feature” column
results_df = results_df.reset_index().rename(columns={'index':'feature'})

results_df 

,feature,coef,pvalue
0,sale_1_month_2022-01,0.204922,1.204187e-97
1,sale_1_month_2022-03,0.168734,1.260925e-104
2,sale_1_month_2022-05,0.163459,3.937253e-115
3,sale_1_month_2022-04,0.160625,7.815193e-122
4,Fur_Solid Gold,0.121178,4.055733e-206
...,...,...,...
120,Clothes_Biker Vest,-0.017616,9.240986e-06
121,Eyes_Closed,-0.021901,2.930317e-05
122,sale_1_month_2021-07,-0.120090,2.324252e-12
123,sale_1_month_2021-06,-0.549294,4.382306e-60


# PartB RandomForest/Decision Tree Regressor for Price_1_sale

In [3]:
import pandas as pd  
import numpy as np 
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import  DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics      import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.preprocessing     import StandardScaler
from sklearn.model_selection import GridSearchCV # Import GridSearchCV
import polars as pl

# read table 
df_table1 = pl.read_csv(
    "df_table1.csv",
    columns=["token_id", "time_1_sale", "price_1_sale"]
).with_columns([
    # Convert UNIX seconds → datetime → format "YYYY-MM"
    pl.col("time_1_sale")
      .cast(pl.Int64)
      .mul(1000)  # Convert seconds to milliseconds
      .cast(pl.Datetime(time_unit="ms"))
      .dt.strftime("%Y-%m")
      .alias("sale_1_month")
])

# read in df_table3 
df_table3 = pl.read_csv("df_table3.csv")


# merge df_table1 and df_table3 on token_id

df_reg1 = df_table1.join(
    df_table3,
    left_on="token_id",
    right_on="token_id",
    how="left",
).with_columns([
    # Convert UNIX seconds → datetime → format "YYYY-MM"
    pl.col("time_1_sale")
      .cast(pl.Int64)
      .mul(1000)  # Convert seconds to milliseconds
      .cast(pl.Datetime(time_unit="ms"))
      .dt.strftime("%Y-%m")
      .alias("sale_1_month")
])

df_partB = df_reg1.drop("time_1_sale")

# categorical columns
cat_cols = ["sale_1_month","Background", "Clothes","Earring", "Eyes","Fur", "Hat","Mouth"]

# One-hot encode categorical features and drop one level per feature (base level)
df_partB = df_partB.to_pandas()
df_partB = pd.get_dummies(df_partB, columns=cat_cols, drop_first=True).copy() # drop_first=True drops the first level of each categorical feature

df_partB.drop(columns=['rarity.rank'], inplace=True) # any analysis on rarity.rank is leakage because it is less likely to stay the same across times


for col in df_partB.columns:
    if df_partB[col].dtype == bool:
        df_partB[col] = df_partB[col].astype("int8")
        
# log+1 on price_1_sale
df_partB['log_price_1'] = np.log1p(df_partB['price_1_sale'])
df_partB.drop(columns=['price_1_sale'], inplace=True) # drop price_1_sale

# X and Y 
drop_cols = ['token_id', 'log_price_1']
X = df_partB.drop(columns=drop_cols)
# save x to csv
X.to_csv("X.csv", index=False)
y = df_partB['log_price_1'].values

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=87)
scalar = StandardScaler()
scalar.fit(X_train)
X_train_scaled = scalar.transform(X_train)
X_test_scale = scalar.transform(X_test)


# Define the parameter grid to search
param_grid = {
    'n_estimators': np.linspace(100, 300, 5, dtype=int), # Reduced number of points for faster search, ensure integer
    'max_depth': [5, 8, 12, 15, 20], # Provide a list of depths to try
    'min_samples_split': [5, 15,20], # Provide a list of values to try
    'min_samples_leaf': [1, 3, 5], # Example: try different leaf sizes
    'max_features': ['sqrt','log2',None, 0.5] # Keep as specified, but in a list
}


# RandomForestRegressor on ten fold cross validation 
rf = RandomForestRegressor(random_state=87, n_jobs=-1)

# Instantiate GridSearchCV
# It performs 10-fold cross-validation internally for each parameter combination
# Scoring 'neg_mean_squared_error' is common for regression, higher (less negative) is better
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           cv=10, n_jobs=-1, scoring='neg_mean_squared_error', verbose=1)

# Fit GridSearchCV to find the best parameters
print("Starting GridSearchCV...")
grid_search.fit(X_train_scaled, y_train)
print("GridSearchCV finished.")

# Get the best estimator found by GridSearchCV
best_rf_model = grid_search.best_estimator_

print(f"Best Parameters found: {grid_search.best_params_}")
print(f"Best cross-validation score (Negative MSE): {grid_search.best_score_:.4f}")


# Predict on the test set using the best model
y_pred = best_rf_model.predict(X_test_scale)

# Evaluate model performance
mse_rf = mean_squared_error(y_test, y_pred)
mae_rf = mean_absolute_error(y_test, y_pred)
rmse_rf = np.sqrt(mse_rf) # Corrected calculation for RMSE
r2_rf = best_rf_model.score(X_test_scale, y_test) # Calculate R-squared

# Calculate MAPE carefully, avoiding division by zero if y_test contains zeros
# Replace zeros or very small values in y_test for MAPE calculation if necessary
# This basic version might error or give misleading results if y_test has zeros.
# Consider using np.log1p transformed values if appropriate or handle zeros explicitly.
mask = y_test != 0
mape_rf = np.mean(np.abs((y_test[mask] - y_pred[mask]) / y_test[mask])) * 100 if np.any(mask) else np.inf


print("\n--- Evaluation on Test Set using Best Model ---")
print(f"RMSE RandomForestRegressor: {rmse_rf:.4f}")
print(f"MAE RandomForestRegressor: {mae_rf:.4f}")
print(f"MSE RandomForestRegressor: {mse_rf:.4f}")
print(f"R^2 RandomForestRegressor: {r2_rf:.4f}")
print(f"MAPE RandomForestRegressor: {mape_rf:.2f}%")

# Feature Importance 
importances = best_rf_model.feature_importances_
feature_names = X.columns # Assuming X is the DataFrame before scaling
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print("\nTop 10 Feature Importances:")
print(feature_importance_df.head(20))

# MAPE due to small values in y_test

C:\Users\Micheli\AppData\Local\Temp\ipykernel_14864\1225810302.py:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_partB['log_price_1'] = np.log1p(df_partB['price_1_sale'])


Starting GridSearchCV...
Fitting 10 folds for each of 900 candidates, totalling 9000 fits
GridSearchCV finished.
Best Parameters found: {'max_depth': 20, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 20, 'n_estimators': 200}
Best cross-validation score (Negative MSE): -0.1385

--- Evaluation on Test Set using Best Model ---
RMSE RandomForestRegressor: 0.3750
MAE RandomForestRegressor: 0.2489
MSE RandomForestRegressor: 0.1406
R^2 RandomForestRegressor: 0.9133
MAPE RandomForestRegressor: 45.64%

Top 10 Feature Importances:
                  Feature  Importance
0    sale_1_month_2021-05    0.532106
1    sale_1_month_2021-06    0.306912
2    sale_1_month_2021-07    0.072597
3    sale_1_month_2021-08    0.014232
136        Fur_Solid Gold    0.008075
138            Fur_Trippy    0.005096
104       Eyes_Blue Beams    0.003392
12   sale_1_month_2022-05    0.003025
59     Clothes_Black Suit    0.003021
11   sale_1_month_2022-04    0.002939
34   sale_1_month_2024-03    0.0028

### Decision Tree

In [ ]:

# Define the parameter grid for Decision Tree
dt_param_grid = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error'], # Common criteria for regression trees
    'max_depth': [5, 10, 15, None], # None means nodes expand until pure or min_samples_split
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 3, 5]
}

# Instantiate the base Decision Tree Regressor model
dt = DecisionTreeRegressor(random_state=87)

# Instantiate GridSearchCV for Decision Tree
# Using 10-fold cross-validation and negative MSE scoring
dt_grid_search = GridSearchCV(estimator=dt, param_grid=dt_param_grid,
                              cv=10, n_jobs=-1, scoring='neg_mean_squared_error', verbose=1)

# Fit GridSearchCV to find the best parameters for Decision Tree
print("\nStarting GridSearchCV for Decision Tree...")
dt_grid_search.fit(X_train_scaled, y_train) # Use the same scaled training data
print("GridSearchCV for Decision Tree finished.")

# Get the best estimator found by GridSearchCV
best_dt_model = dt_grid_search.best_estimator_

print(f"\nBest Parameters found for Decision Tree: {dt_grid_search.best_params_}")
print(f"Best cross-validation score (Negative MSE) for Decision Tree: {dt_grid_search.best_score_:.4f}")

# --- Evaluate the Best Decision Tree Model on the Test Set ---

# Predict on the test set using the best decision tree model
y_pred_dt = best_dt_model.predict(X_test_scale) # Use the same scaled test data

# Evaluate model performance
mse_dt = mean_squared_error(y_test, y_pred_dt)
mae_dt = mean_absolute_error(y_test, y_pred_dt)
rmse_dt = np.sqrt(mse_dt)
r2_dt = best_dt_model.score(X_test_scale, y_test) # Calculate R-squared

# Calculate MAPE carefully
mask = y_test != 0
mape_dt = np.mean(np.abs((y_test[mask] - y_pred_dt[mask]) / y_test[mask])) * 100 if np.any(mask) else np.inf

print("\n--- Evaluation on Test Set using Best Decision Tree Model ---")
print(f"RMSE DecisionTreeRegressor: {rmse_dt:.4f}")
print(f"MAE DecisionTreeRegressor: {mae_dt:.4f}")
print(f"MSE DecisionTreeRegressor: {mse_dt:.4f}")
print(f"R^2 DecisionTreeRegressor: {r2_dt:.4f}")
print(f"MAPE DecisionTreeRegressor: {mape_dt:.2f}%")

# Feature Importance for Decision Tree
dt_importances = best_dt_model.feature_importances_
dt_feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': dt_importances})
dt_feature_importance_df = dt_feature_importance_df.sort_values(by='Importance', ascending=False)

print("\nTop 10 Feature Importances (Decision Tree):")
print(dt_feature_importance_df.head(10))


Starting GridSearchCV for Decision Tree...
Fitting 10 folds for each of 108 candidates, totalling 1080 fits
GridSearchCV for Decision Tree finished.

Best Parameters found for Decision Tree: {'criterion': 'absolute_error', 'max_depth': 15, 'min_samples_leaf': 3, 'min_samples_split': 10}
Best cross-validation score (Negative MSE) for Decision Tree: -0.1623

--- Evaluation on Test Set using Best Decision Tree Model ---
RMSE DecisionTreeRegressor: 0.4190
MAE DecisionTreeRegressor: 0.2663
MSE DecisionTreeRegressor: 0.1756
R^2 DecisionTreeRegressor: 0.8917
MAPE DecisionTreeRegressor: 43.23%

Top 10 Feature Importances (Decision Tree):
                  Feature  Importance
0    sale_1_month_2021-05    0.445521
1    sale_1_month_2021-06    0.318649
2    sale_1_month_2021-07    0.096155
3    sale_1_month_2021-08    0.029197
136        Fur_Solid Gold    0.008639
138            Fur_Trippy    0.007546
34   sale_1_month_2024-03    0.005756
12   sale_1_month_2022-05    0.005526
8    sale_1_month_2

# 08/29 Model Comparison

In [4]:
# =========================
# 1) Imports
# =========================
import pandas as pd
import numpy as np
import polars as pl

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

# xgboost
from xgboost import XGBRegressor

RNG = 87

# =========================
# 2) Data load & feature engineering (same logic you had)
# =========================
# df_table1
df_table1 = pl.read_csv(
    "df_table1.csv",
    columns=["token_id", "time_1_sale", "price_1_sale"]
).with_columns([
    pl.col("time_1_sale")
      .cast(pl.Int64)
      .mul(1000)
      .cast(pl.Datetime(time_unit="ms"))
      .dt.strftime("%Y-%m")
      .alias("sale_1_month")
])

# df_table3
df_table3 = pl.read_csv("df_table3.csv")

# merge
df_reg1 = df_table1.join(
    df_table3,
    left_on="token_id",
    right_on="token_id",
    how="left",
).with_columns([
    pl.col("time_1_sale")
      .cast(pl.Int64)
      .mul(1000)
      .cast(pl.Datetime(time_unit="ms"))
      .dt.strftime("%Y-%m")
      .alias("sale_1_month")
])

# drop raw time (already converted)
df_partB = df_reg1.drop("time_1_sale")

# categorical columns to one-hot
cat_cols = ["sale_1_month","Background","Clothes","Earring","Eyes","Fur","Hat","Mouth"]

# one-hot encode (drop first to avoid full rank)
df_partB = df_partB.to_pandas()
df_partB = pd.get_dummies(df_partB, columns=cat_cols, drop_first=True).copy()

# remove leakage
if 'rarity.rank' in df_partB.columns:
    df_partB.drop(columns=['rarity.rank'], inplace=True)

# compact bools
for col in df_partB.columns:
    if df_partB[col].dtype == bool:
        df_partB[col] = df_partB[col].astype("int8")

# target on log scale
df_partB['log_price_1'] = np.log1p(df_partB['price_1_sale'])
df_partB.drop(columns=['price_1_sale'], inplace=True)

# features / target
X = df_partB.drop(columns=['token_id','log_price_1'])
y = df_partB['log_price_1'].values
feature_names = X.columns.tolist()

# =========================
# 3) Train / test split
# =========================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RNG
)

# NOTE: Tree-based models don't need scaling, so we skip StandardScaler.

# =========================
# 4) Model zoo & grids
# =========================
models = {
    "RandomForest": RandomForestRegressor(random_state=RNG, n_jobs=-1),
    "XGBoost": XGBRegressor(
        random_state=RNG,
        n_estimators=300,
        tree_method="hist",      # fast & good default
        n_jobs=-1
    ),
    "GradientBoosting": GradientBoostingRegressor(random_state=RNG),
    "AdaBoost": AdaBoostRegressor(
        estimator=DecisionTreeRegressor(max_depth=3, random_state=RNG),
        random_state=RNG
    ),
}

param_grids = {
    "RandomForest": {
        "n_estimators": [100, 150, 200, 250, 300],
        "max_depth": [5, 8, 12, 15, 20],
        "min_samples_split": [5, 15, 20],
        "min_samples_leaf": [1, 3, 5],
        "max_features": ["sqrt", "log2", None, 0.5],
    },
    "XGBoost": {
        "n_estimators": [200, 350, 500],
        "max_depth": [4, 6, 8],
        "learning_rate": [0.03, 0.06, 0.1],
        "subsample": [0.7, 0.9, 1.0],
        "colsample_bytree": [0.7, 0.9, 1.0],
        "reg_lambda": [1.0, 3.0, 6.0],
    },
    "GradientBoosting": {
        "n_estimators": [200, 400],
        "learning_rate": [0.05, 0.1],
        "max_depth": [2, 3, 4],
        "subsample": [0.7, 0.9, 1.0],
        "min_samples_leaf": [1, 3, 5],
    },
    "AdaBoost": {
        "n_estimators": [200, 400, 600],
        "learning_rate": [0.05, 0.1, 0.2],
        # tune the base tree depth
        "estimator__max_depth": [2, 3, 4],
    },
}

# =========================
# 5) Helper: metrics on log-scale and original price-scale
# =========================
def evaluate(y_true_log, y_pred_log, label):
    # log-scale metrics
    mse_log = mean_squared_error(y_true_log, y_pred_log)
    rmse_log = np.sqrt(mse_log)
    mae_log = mean_absolute_error(y_true_log, y_pred_log)
    r2_log = r2_score(y_true_log, y_pred_log)

    # back-transform to original price scale
    y_true = np.expm1(y_true_log)
    y_pred = np.expm1(y_pred_log)

    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    # MAPE safe (avoid div by zero)
    mask = np.abs(y_true) > 1e-12
    mape = (np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])).mean() * 100 if mask.any() else np.inf

    row = dict(
        Model=label,
        RMSE_log=rmse_log, MAE_log=mae_log, R2_log=r2_log,
        RMSE=rmse, MAE=mae, R2=r2, MAPE_pct=mape
    )
    return row

# =========================
# 6) GridSearchCV for each model
# =========================
results = []
best_models = {}

for name, est in models.items():
    print(f"\n=== {name} : GridSearchCV ===")
    grid = param_grids[name]
    gs = GridSearchCV(
        estimator=est,
        param_grid=grid,
        scoring="neg_mean_squared_error",  # on log target
        cv=10,
        n_jobs=-1,
        verbose=1
    )
    gs.fit(X_train, y_train)
    print(f"Best params: {gs.best_params_}")
    print(f"Best CV (neg MSE): {gs.best_score_:.6f}")

    best = gs.best_estimator_
    best_models[name] = best

    # Evaluate on test
    y_pred_log = best.predict(X_test)
    row = evaluate(y_test, y_pred_log, name)
    results.append({**row, "BestParams": gs.best_params_})

# =========================
# 7) Results table
# =========================
results_df = pd.DataFrame(results).sort_values(by="RMSE")  # sort by RMSE on original scale
print("\n=== Test-set comparison (sorted by RMSE on original scale) ===")
print(results_df[["Model","RMSE","MAE","R2","MAPE_pct","RMSE_log","MAE_log","R2_log"]])

# Save detailed results with params
results_df.to_csv("model_comparison_results.csv", index=False)

# =========================
# 8) Feature importances (where available)
# =========================
def dump_feature_importance(model, feature_names, topn=20, label="Model"):
    if hasattr(model, "feature_importances_"):
        fi = pd.DataFrame({
            "Feature": feature_names,
            "Importance": model.feature_importances_
        }).sort_values("Importance", ascending=False)
        print(f"\nTop {topn} feature importances — {label}")
        print(fi.head(topn))
        fi.to_csv(f"feature_importance_{label}.csv", index=False)
    elif isinstance(model, XGBRegressor):
        # xgboost also exposes .feature_importances_
        fi = pd.DataFrame({
            "Feature": feature_names,
            "Importance": model.feature_importances_
        }).sort_values("Importance", ascending=False)
        print(f"\nTop {topn} feature importances — {label} (XGB)")
        print(fi.head(topn))
        fi.to_csv(f"feature_importance_{label}.csv", index=False)
    else:
        print(f"\n{label} does not expose feature_importances_.")

for name, mdl in best_models.items():
    dump_feature_importance(mdl, feature_names, topn=20, label=name)

# =========================
# 9) Optional: persist best models (pickle)
# =========================
import joblib
for name, mdl in best_models.items():
    joblib.dump(mdl, f"best_{name}.joblib")
print("\nSaved best models to disk (best_*.joblib).")



=== RandomForest : GridSearchCV ===
Fitting 10 folds for each of 900 candidates, totalling 9000 fits


/var/folders/zd/cp61vmzd4179rc6wz_57mvv00000gn/T/ipykernel_1780/304746586.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_partB['log_price_1'] = np.log1p(df_partB['price_1_sale'])
/Users/miultra/Documents/Tech/Miniconda/miniconda3/envs/NFT_25/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/miultra/Documents/Tech/Miniconda/miniconda3/envs/NFT_25/lib/python3.11/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best params: {'max_depth': 20, 'max_features': 0.5, 'min_samples_leaf': 1, 'min_samples_split': 20, 'n_estimators': 250}
Best CV (neg MSE): -0.129936

=== XGBoost : GridSearchCV ===
Fitting 10 folds for each of 729 candidates, totalling 7290 fits


/Users/miultra/Documents/Tech/Miniconda/miniconda3/envs/NFT_25/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best params: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 500, 'reg_lambda': 6.0, 'subsample': 0.9}
Best CV (neg MSE): -0.109858

=== GradientBoosting : GridSearchCV ===
Fitting 10 folds for each of 108 candidates, totalling 1080 fits


/Users/miultra/Documents/Tech/Miniconda/miniconda3/envs/NFT_25/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best params: {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_leaf': 5, 'n_estimators': 400, 'subsample': 0.7}
Best CV (neg MSE): -0.108496

=== AdaBoost : GridSearchCV ===
Fitting 10 folds for each of 27 candidates, totalling 270 fits


/Users/miultra/Documents/Tech/Miniconda/miniconda3/envs/NFT_25/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best params: {'estimator__max_depth': 4, 'learning_rate': 0.05, 'n_estimators': 200}
Best CV (neg MSE): -0.222447

=== Test-set comparison (sorted by RMSE on original scale) ===
              Model         RMSE        MAE        R2      MAPE_pct  RMSE_log  \
1           XGBoost  1857.680426  45.813368  0.001384  1.376769e+07  0.371412   
2  GradientBoosting  1857.832161  45.787736  0.001221  1.299586e+07  0.371958   
0      RandomForest  1858.420370  46.687269  0.000588  9.668128e+06  0.403981   
3          AdaBoost  1858.771495  49.122239  0.000210  7.210167e+06  0.516282   

    MAE_log    R2_log  
1  0.228468  0.908851  
2  0.228223  0.908583  
0  0.246026  0.892164  
3  0.353107  0.823877  

Top 20 feature importances — RandomForest
                         Feature  Importance
0           sale_1_month_2021-05    0.486728
1           sale_1_month_2021-06    0.253033
2           sale_1_month_2021-07    0.062577
3           sale_1_month_2021-08    0.053849
8           sale_1_month_202

# Bug fix

In [ ]:
# =========================
# 1) Imports & setup
# =========================
import numpy as np
import pandas as pd
import polars as pl
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedKFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.inspection import permutation_importance
from xgboost import XGBRegressor

RNG = 87
np.random.seed(RNG)

# =========================
# 2) Load raw & minimal FE (NO encoding yet)
# =========================
# Read df_table1 with month feature
df_table1 = pl.read_csv(
    "df_table1.csv",
    columns=["token_id", "time_1_sale", "price_1_sale"]
).with_columns([
    pl.col("time_1_sale").cast(pl.Int64).mul(1000).cast(pl.Datetime(time_unit="ms")).dt.strftime("%Y-%m").alias("sale_1_month")
])

# Read df_table3
df_table3 = pl.read_csv("df_table3.csv")

# Merge on token_id
df_reg1 = df_table1.join(df_table3, left_on="token_id", right_on="token_id", how="left")

# Keep a pandas copy with raw categoricals (no encoding yet)
raw = df_reg1.drop("time_1_sale").to_pandas()

# (Optional) remove leakage columns if present
if 'rarity.rank' in raw.columns:
    raw = raw.drop(columns=['rarity.rank'])

# =========================
# 3) Split BEFORE one-hot to avoid leakage
# =========================
idx_train, idx_test = train_test_split(raw.index, test_size=0.2, random_state=RNG)
train_df = raw.loc[idx_train].copy()
test_df  = raw.loc[idx_test].copy()

# Target (log1p for training)
y_train = np.log1p(train_df['price_1_sale']).values
y_test  = np.log1p(test_df['price_1_sale']).values

# Drop columns not for X
drop_cols = ['token_id', 'price_1_sale']
cat_cols = ["sale_1_month", "Background", "Clothes", "Earring", "Eyes", "Fur", "Hat", "Mouth"]

# One-hot on TRAIN only
X_train = pd.get_dummies(
    train_df.drop(columns=drop_cols),
    columns=cat_cols, drop_first=True
)

# One-hot TEST with train's columns (unknown levels → 0)
X_test = pd.get_dummies(
    test_df.drop(columns=drop_cols),
    columns=cat_cols, drop_first=True
)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

feature_names = X_train.columns.tolist()

# =========================
# 4) Model zoo & param grids
# =========================
models = {
    "RandomForest": RandomForestRegressor(random_state=RNG, n_jobs=-1),
    "XGBoost": XGBRegressor(random_state=RNG, n_estimators=300, tree_method="hist", n_jobs=-1),
    "GradientBoosting": GradientBoostingRegressor(random_state=RNG),
    "AdaBoost": AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=3, random_state=RNG), random_state=RNG),
}

param_grids = {
    "RandomForest": {
        "n_estimators": [100, 150, 200, 250, 300],
        "max_depth": [5, 8, 12, 15, 20],
        "min_samples_split": [5, 15, 20],
        "min_samples_leaf": [1, 3, 5],
        "max_features": ["sqrt", "log2", None, 0.5],
    },
    "XGBoost": {
        "n_estimators": [200, 350, 500],
        "max_depth": [4, 6, 8],
        "learning_rate": [0.03, 0.06, 0.1],
        "subsample": [0.7, 0.9, 1.0],
        "colsample_bytree": [0.7, 0.9, 1.0],
        "reg_lambda": [1.0, 3.0, 6.0],
    },
    "GradientBoosting": {
        "n_estimators": [200, 400],
        "learning_rate": [0.05, 0.1],
        "max_depth": [2, 3, 4],
        "subsample": [0.7, 0.9, 1.0],
        "min_samples_leaf": [1, 3, 5],
    },
    "AdaBoost": {
        "n_estimators": [200, 400, 600],
        "learning_rate": [0.05, 0.1, 0.2],
        "estimator__max_depth": [2, 3, 4],
    },
}

# =========================
# 5) Metrics helpers (log + original), sMAPE, diagnostics
# =========================
def _eval_block(y_true_log, y_pred_log, eps=1e-12):
    rmse_log = np.sqrt(mean_squared_error(y_true_log, y_pred_log))
    mae_log  = mean_absolute_error(y_true_log, y_pred_log)
    r2_log   = r2_score(y_true_log, y_pred_log)

    y_true = np.expm1(y_true_log)
    y_pred = np.expm1(y_pred_log)

    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)

    denom = np.where(np.abs(y_true) < eps, eps, np.abs(y_true))
    mape = np.mean(np.abs((y_true - y_pred) / denom)) * 100

    smape = np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + eps)) * 100

    return dict(RMSE_log=rmse_log, MAE_log=mae_log, R2_log=r2_log,
                RMSE=rmse, MAE=mae, R2=r2, MAPE_pct=mape, sMAPE_pct=smape)

def eval_train_test(name, model, Xtr, ytr, Xte, yte):
    yhat_tr = model.predict(Xtr)
    yhat_te = model.predict(Xte) 
    tr = _eval_block(ytr, yhat_tr)
    te = _eval_block(yte, yhat_te)
    print(f"\n[{name}] Train vs Test (log-scale): "
          f"Train R2_log={tr['R2_log']:.4f}, Test R2_log={te['R2_log']:.4f}, "
          f"Train RMSE_log={tr['RMSE_log']:.4f}, Test RMSE_log={te['RMSE_log']:.4f}")
    return tr, te

def cv_report(name, est, X_tr, y_tr, rng=RNG):
    rkf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=rng)
    cv_mse = -cross_val_score(est, X_tr, y_tr, scoring="neg_mean_squared_error", cv=rkf, n_jobs=-1)
    cv_rmse = np.sqrt(cv_mse)
    print(f"[{name}] RepeatedKFold CV RMSE (log): mean={cv_rmse.mean():.4f}, std={cv_rmse.std():.4f}")

def perm_importance_test(name, model, Xte, yte, features, topn=15): 
    pi = permutation_importance(model, Xte, yte, n_repeats=10, random_state=RNG, n_jobs=-1,
                                scoring="neg_mean_squared_error")
    df = pd.DataFrame({
        "Feature": features,
        "MeanDecreaseMSE_log": pi.importances_mean,
        "Std": pi.importances_std
    }).sort_values("MeanDecreaseMSE_log", ascending=False)
    print(f"\nPermutation importance on TEST (log MSE) — {name} (top {topn}):")
    print(df.head(topn))
    df.to_csv(f"perm_importance_{name}.csv", index=False)

def impurity_importance(name, model, features, topn=20): #1
    if hasattr(model, "feature_importances_"):
        fi = pd.DataFrame({"Feature": features, "Importance": model.feature_importances_}) \
             .sort_values("Importance", ascending=False)
        print(f"\nTop {topn} impurity feature importances — {name}")
        print(fi.head(topn))
        fi.to_csv(f"feature_importance_{name}.csv", index=False)

# =========================
# 6) GridSearchCV → fit → diagnostics → collect results
# =========================
results = []
best_models = {}

for name, est in models.items():
    print(f"\n=== {name} : GridSearchCV (cv=10) ===")
    gs = GridSearchCV(
        estimator=est,
        param_grid=param_grids[name],
        scoring="neg_mean_squared_error",   # on LOG target
        cv=10, n_jobs=-1, verbose=1
    )
    gs.fit(X_train, y_train)
    print(f"Best params: {gs.best_params_}")
    print(f"Best CV (neg MSE): {gs.best_score_:.6f}")

    best = gs.best_estimator_
    best_models[name] = best

    # Train vs Test metrics (overfitting check)
    tr, te = eval_train_test(name, best, X_train, y_train, X_test, y_test)

    # Repeated K-Fold CV robustness on TRAIN 5
    cv_report(name, best, X_train, y_train)

    # Permutation importance on TEST (less biased)
    perm_importance_test(name, best, X_test, y_test, feature_names)

    # Impurity-based FI (if available) 0
    impurity_importance(name, best, feature_names)

    # Store test-row for comparison table
    results.append({
        "Model": name,
        **te,
        "BestParams": gs.best_params_
    })

# =========================
# 7) Summary table & persistence
# =========================
results_df = pd.DataFrame(results).sort_values(by="RMSE")  # lowest RMSE (original scale) first
print("\n=== Test-set comparison (sorted by RMSE on original scale) ===")
cols = ["Model","RMSE","MAE","R2","MAPE_pct","sMAPE_pct","RMSE_log","MAE_log","R2_log","BestParams"]
print(results_df[cols])
results_df.to_csv("model_comparison_results.csv", index=False)

# Save fitted best models
for name, mdl in best_models.items():
    joblib.dump(mdl, f"best_{name}.joblib")
print("\nSaved best models to disk (best_*.joblib).")



=== RandomForest : GridSearchCV (cv=10) ===
Fitting 10 folds for each of 900 candidates, totalling 9000 fits


/Users/miultra/Documents/Tech/Miniconda/miniconda3/envs/NFT_25/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/miultra/Documents/Tech/Miniconda/miniconda3/envs/NFT_25/lib/python3.11/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best params: {'max_depth': 20, 'max_features': 0.5, 'min_samples_leaf': 1, 'min_samples_split': 20, 'n_estimators': 250}
Best CV (neg MSE): -0.429398

[RandomForest] Train vs Test (log-scale): Train R2_log=0.7640, Test R2_log=0.7063, Train RMSE_log=0.6168, Test RMSE_log=0.6667
[RandomForest] RepeatedKFold CV RMSE (log): mean=0.6564, std=0.0289


/Users/miultra/Documents/Tech/Miniconda/miniconda3/envs/NFT_25/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(



Permutation importance on TEST (log MSE) — RandomForest (top 15):
                 Feature  MeanDecreaseMSE_log       Std
2   sale_1_month_2021-08             0.395327  0.015694
11  sale_1_month_2022-05             0.287492  0.024823
0   sale_1_month_2021-06             0.164565  0.009010
9   sale_1_month_2022-03             0.159964  0.012892
1   sale_1_month_2021-07             0.154020  0.012684
7   sale_1_month_2022-01             0.141445  0.004943
6   sale_1_month_2021-12             0.128285  0.008716
10  sale_1_month_2022-04             0.125674  0.014408
3   sale_1_month_2021-09             0.101800  0.008486
8   sale_1_month_2022-02             0.087925  0.009957
12  sale_1_month_2022-06             0.078028  0.007057
5   sale_1_month_2021-11             0.062716  0.008281
22  sale_1_month_2023-04             0.060421  0.006920
18  sale_1_month_2022-12             0.053200  0.005920
17  sale_1_month_2022-11             0.034210  0.003867

Top 20 impurity feature importances 

...

Best params: {'learning_rate': 0.1, 'max_depth': 4, 'min_samples_leaf': 3, 'n_estimators': 400, 'subsample': 0.7}
Best CV (neg MSE): -0.117783

[GradientBoosting] Train vs Test (log-scale): Train R2_log=0.9416, Test R2_log=0.9066, Train RMSE_log=0.3067, Test RMSE_log=0.3760
[GradientBoosting] RepeatedKFold CV RMSE (log): mean=0.3458, std=0.0218


/Users/miultra/Documents/Tech/Miniconda/miniconda3/envs/NFT_25/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(



Permutation importance on TEST (log MSE) — GradientBoosting (top 15):
                 Feature  MeanDecreaseMSE_log       Std
2   sale_1_month_2021-08             0.431113  0.010188
11  sale_1_month_2022-05             0.325487  0.016051
1   sale_1_month_2021-07             0.187614  0.007125
0   sale_1_month_2021-06             0.183667  0.004058
9   sale_1_month_2022-03             0.179243  0.008872
6   sale_1_month_2021-12             0.153199  0.004374
7   sale_1_month_2022-01             0.151513  0.003821
10  sale_1_month_2022-04             0.137409  0.009009
3   sale_1_month_2021-09             0.124467  0.005522
8   sale_1_month_2022-02             0.115148  0.003482
14  sale_1_month_2022-08             0.108670  0.003635
12  sale_1_month_2022-06             0.087196  0.003642
5   sale_1_month_2021-11             0.071890  0.002389
22  sale_1_month_2023-04             0.066852  0.002688
18  sale_1_month_2022-12             0.066094  0.001460

Top 20 impurity feature importan

/Users/miultra/Documents/Tech/Miniconda/miniconda3/envs/NFT_25/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best params: {'estimator__max_depth': 4, 'learning_rate': 0.05, 'n_estimators': 200}
Best CV (neg MSE): -1.647785

[AdaBoost] Train vs Test (log-scale): Train R2_log=-0.0389, Test R2_log=-0.0985, Train RMSE_log=1.2940, Test RMSE_log=1.2894
[AdaBoost] RepeatedKFold CV RMSE (log): mean=1.2801, std=0.0200

Permutation importance on TEST (log MSE) — AdaBoost (top 15):
                  Feature  MeanDecreaseMSE_log       Std
2    sale_1_month_2021-08             0.054966  0.004220
11   sale_1_month_2022-05             0.022391  0.003492
7    sale_1_month_2022-01             0.017235  0.003757
9    sale_1_month_2022-03             0.012902  0.002384
10   sale_1_month_2022-04             0.010035  0.001526
6    sale_1_month_2021-12             0.005434  0.001671
12   sale_1_month_2022-06             0.003322  0.000515
135        Fur_Solid Gold             0.002723  0.002106
17   sale_1_month_2022-11             0.002004  0.000296
8    sale_1_month_2022-02             0.001674  0.001654
5    s

# Padding Space figures fixes

In [3]:
import pandas as pd 

df1 = pd.read_csv('df_table1.csv', usecols=['token_id'])
df3 = pd.read_csv('df_table3.csv')

df = df1.merge(df3, on='token_id', how='left')

df.drop(columns=['rarity.rank'], inplace=True)
df

,token_id,Background,Clothes,Earring,Eyes,Fur,Hat,Mouth
0,9252,Purple,Smoking Jacket,NaN,Angry,Pink,Bayc Hat Red,Tongue Out
1,1448,Yellow,Puffy Vest,Silver Stud,Bored,Tan,Fez,Jovial
2,8444,Orange,Biker Vest,Silver Hoop,Sad,Blue,NaN,Small Grin
3,1741,Aquamarine,Bone Tee,NaN,Zombie,Black,Army Hat,Small Grin
4,8025,Blue,Vietnam Jacket,NaN,Sleepy,Dark Brown,NaN,Bored Unshaven Cigarette
...,...,...,...,...,...,...,...,...
9249,6921,New Punk Blue,Hawaiian,Silver Stud,Sunglasses,Black,NaN,Bored Cigarette
9250,2340,Aquamarine,NaN,NaN,Zombie,Gray,Beanie,Jovial
9251,2478,Army Green,Prison Jumpsuit,NaN,Sad,Brown,Fez,Rage
9252,6165,Gray,Bone Tee,NaN,Coins,Tan,Girl's Hair Short,Bored Unshaven


In [ ]:

import pandas as pd

# load & merge (adapt to how your notebook reads these files)
df1 = pd.read_csv('df_table1.csv', usecols=['token_id'])
df3 = pd.read_csv('df_table3.csv')

df = df1.merge(df3, on='token_id', how='left')

# drop if present
if 'rarity.rank' in df.columns:
    df = df.drop(columns=['rarity.rank'])

# normalize Fur values
df['Fur'] = df['Fur'].astype(str).str.strip()

# columns that must be identical between matched rows (everything except token_id and Fur)
cols_other = [c for c in df.columns if c not in ['token_id', 'Fur']]

if not cols_other:
    raise ValueError("No columns left to compare — check your dataframe columns.")

# Method A: groupby filter (keeps whole groups where Fur values are exactly Black & Solid Gold)
groups_exact = df.groupby(cols_other).filter(
    lambda g: set(g['Fur'].dropna().unique()) == {'Black', 'Solid Gold'}
)

# Aggregate token_ids and fur values per matched group
grouped = (
    groups_exact
    .groupby(cols_other, dropna=False)
    .agg(token_ids=('token_id', lambda s: list(s.unique())),
         furs=('Fur', lambda s: sorted(set(s.dropna().tolist()))))
    .reset_index()
)

print(f"Groups with only Black & Solid Gold (and identical other columns): {len(grouped)}")
if len(grouped):
    display(grouped.head())

# Method B: pairwise merge (fast, returns explicit Black-SolidGold token_id pairs)
df_black = df[df['Fur'].eq('Black')].copy()
df_sg = df[df['Fur'].eq('Solid Gold')].copy()

pairs = df_black.merge(df_sg, on=cols_other, suffixes=('_black', '_sg'))
# ensure different token_ids (should be, but safe)
pairs = pairs[pairs['token_id_black'] != pairs['token_id_sg']].drop_duplicates(
    subset=['token_id_black', 'token_id_sg']
).reset_index(drop=True)

print(f"Pair rows found (Black vs Solid Gold): {len(pairs)}")
if len(pairs):
    display(pairs[['token_id_black','token_id_sg']].head())

# Flatten unique token_ids involved
token_ids = pd.unique(pairs[['token_id_black','token_id_sg']].values.ravel())
print(f"Total unique token_ids involved: {len(token_ids)}")

# Optional: save results
pairs[['token_id_black','token_id_sg']].to_csv('token_pairs_black_solidgold.csv', index=False)
pd.Series(token_ids, name='token_id').to_csv('token_ids_black_solidgold.csv', index=False)

# return variables for interactive use
grouped, pairs, token_ids
#

Groups with only Black & Solid Gold (and identical other columns): 0
Pair rows found (Black vs Solid Gold): 0
Total unique token_ids involved: 0


(Empty DataFrame
 Columns: [Background, Clothes, Earring, Eyes, Hat, Mouth, token_ids, furs]
 Index: [],
 Empty DataFrame
 Columns: [token_id_black, Background, Clothes, Earring, Eyes, Fur_black, Hat, Mouth, token_id_sg, Fur_sg]
 Index: [],
 array([], dtype=int64))

In [2]:

import pandas as pd 

df1 = pd.read_csv('df_table1.csv', usecols=['token_id']).set_index('token_id')
df3 = pd.read_csv('df_table3.csv').set_index('token_id')

df = df1.join(df3, how='left')  # merge on index

# clean up
if 'rarity.rank' in df.columns:
    df = df.drop(columns=['rarity.rank'])

# normalize Fur values
df['Fur'] = df['Fur'].astype(str).str.strip()

# columns that must be identical (everything except Fur)
cols_other = [c for c in df.columns if c != 'Fur']
if not cols_other:
    raise ValueError("No columns left to compare — check your dataframe columns.")

# =========================
# Method B: explicit pairs
# =========================
# Create pairs of rows with identical 'other' columns but different Fur
tmp = df.reset_index()  # bring token_id out for pairing
left  = tmp.rename(columns={'Fur': 'Fur_left',  'token_id': 'token_id_left'})
right = tmp.rename(columns={'Fur': 'Fur_right', 'token_id': 'token_id_right'})

pairs = left.merge(
    right,
    on=cols_other,        # identical across all non-Fur columns
    how='inner',
    suffixes=('_left','_right')
)

# keep pairs with different Fur, and dedupe symmetric pairs
pairs = pairs[pairs['Fur_left'] != pairs['Fur_right']]
pairs = pairs[pairs['token_id_left'] < pairs['token_id_right']].copy()

print(f"Explicit pairs (same others, different Fur): {len(pairs)}")
if len(pairs):
    display(pairs[['token_id_left','Fur_left','token_id_right','Fur_right']])

# Unique token_ids involved
token_ids = pd.unique(
    pd.concat([pairs['token_id_left'], pairs['token_id_right']], ignore_index=True)
)
print(f"Total unique token_ids involved: {len(token_ids)}")


Explicit pairs (same others, different Fur): 67


,token_id_left,Fur_left,token_id_right,Fur_right
33,6366,Golden Brown,9604,Cheetah
316,821,Cheetah,938,Black
317,821,Cheetah,8560,Robot
390,3131,Dark Brown,8265,Gray
391,3131,Dark Brown,6327,Red
...,...,...,...,...
9006,6327,Red,8265,Gray
9156,6013,Brown,8512,Cream
9164,2632,Black,5980,Brown
9275,1416,Gray,7558,Tan


Total unique token_ids involved: 119
